In [ ]:
import os, getpass
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# _set_env("ANTHROPIC_API_KEY")
# _set_env("NEO4J_URI")
# _set_env("NEO4J_USERNAME")
# _set_env("NEO4J_PASSWORD")

In [ ]:
from dotenv import load_dotenv  
from langchain.chains import GraphCypherQAChain  
from langchain_community.graphs import Neo4jGraph  
from langchain_core.documents import Document  
from langchain_experimental.graph_transformers import LLMGraphTransformer  
from langchain_anthropic import ChatAnthropic
  
load_dotenv()  
  
llm = ChatAnthropic(temperature=0, model_name="claude-3-haiku-20240307")  
  
llm_transformer = LLMGraphTransformer(llm=llm)

In [ ]:
from langchain_community.chains.graph_qa.prompts import CYPHER_GENERATION_PROMPT

graph = Neo4jGraph()

def query_graph(graph, query):  
    chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True, validate_cypher=True, allow_dangerous_requests = True, top_k = 5, cypher_prompt=CYPHER_GENERATION_PROMPT )  
    response = chain.invoke({"query": query})  
    return response

In [ ]:
response = query_graph(graph, "What health problems did Nicola Tesla have?")
print(response)

In [ ]:
text = """Medical information about Nicola Tesla:
Mental health: Tesla was known to have obsessive-compulsive tendencies. He was particular about cleanliness and had specific routines and habits.
Sleep patterns: Tesla reportedly slept very little, claiming to sleep only about two hours per night. However, he was said to occasionally nap during the day.
Diet: In his later years, Tesla became a vegetarian. He was very particular about his food and would often only eat honey, milk, and vegetables.
Celibacy: Tesla practiced lifelong celibacy, believing it helped him focus on his work.
Phobias: He had several phobias, including a fear of germs (mysophobia) and an aversion to pearls and earrings on women.
Physical health: Despite his eccentric habits, Tesla lived to be 86 years old, which was well above the average life expectancy for his time.
Possible neurological condition: Some modern researchers have speculated that Tesla may have had a neurological condition, possibly a high-functioning form of autism or Asperger's syndrome, based on his behaviors and work patterns.
Vision and auditory experiences: Tesla claimed to have vivid flashes of light accompanied by visions, particularly in his younger years. He also reported having acute hearing sensitivity.
Later life health issues: In his later years, Tesla was known to have become increasingly frail and had suffered at least one nervous breakdown.
"""

In [ ]:
def add_knowledge_to_graph( text):
    documents = [Document(page_content=text)]
    graph_documents = llm_transformer.convert_to_graph_documents(documents)
    graph.add_graph_documents(graph_documents)

    graph.refresh_schema()  # Refreshes the Neo4j graph schema information.
    
add_knowledge_to_graph(text)

In [ ]:
response = query_graph(graph, "What health problems did Nicola Tesla have?")
print(response)